## III. Theme

In [3]:
import os
import re
import csv
from urllib import urlopen
import sys
import csv
from collections import Counter
from __future__ import division

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [4]:
# load data
recs = []
dir = '.'
with open("../Data/upr-orig.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
print len(recs) # 42013 
recs[21]

39329


{'Action_Verb': "['continue']",
 'Decision': 'support',
 'From': 'Canada',
 'From_COW': 'CAN',
 'Institution': "['CAT', 'CRC']",
 'Report_Year': '2008',
 'Session': '1',
 'Session_Year': '2008',
 'Text': '64.12 To continue its efforts to improve the situation of children in detention as recommended by the Committee against Torture and the Committee on the Rights of the Child. (Canada, Italy, Slovenia);',
 'To': 'Argentina',
 'To_COW': 'ARG',
 'UID': 'ARG-CAN-1-22'}

In [5]:
# strip extra whitespace on everything
for i in recs:
    i['Text']=' '.join(i['Text'].split())

In [6]:
# remove stop words function
stopwords = ['of','on','all','the','a','for','to','and','to','against','from','in','or']

def noStop(in_text):
    '''
    this function inputs a text, lowers it, and removes stop words
    '''
    text = in_text.lower().split()
    for word in stopwords:
        text = [x for x in text if x != word]
    return(' '.join(text))

In [7]:
noStop(recs[1]['Text'])

'69.7 that algeria continue strengthen its efforts field economic, social cultural rights, particular field health (cuba). );'

In [8]:
## Make Theme Dictionary

In [31]:
keywords = {}

# Physical Integrity Rights

keywords['extra-judicial'] = ['extrajudicial', 'political killing', 'deprivation life']

keywords['torture'] = ['torture', 'inhuman degrading treatment', 'stoning', 'amputation', \
                       'caning', 'whipping', 'waterboard', 'inhumane', 'forced confession']

keywords['pol-imprisonment'] = ['political prison', 'prisoner of conscience']

keywords['disappearances'] = ['disappear']

keywords['trafficking'] = ['slavery', 'forced labor', 'forced labour', 'traffick', 'slave', \
                           'forced prostitution','sale children']

keywords['prisoners'] = ['prison','arbitrary detention', 'interrog', 'incarceration', 'solitary confinement', \
                         'detained', 'detent', 'arbitrary investigation', 'access lawyer' , 'criminal investigations', \
                        'police custody', 'preventive custody', 'pretrial custody', 'access council', 'detainee']

keywords['death-pen'] = ['death penalty', 'capital punishment', 'death sentence', 'executions',\
                         'capital sentence']

keywords['sex-violence'] = ['domestic violence', 'sexual violence', 'rape', 'domestic abuse', 'sexual abuse', \
                            'gender-based violence', 'violence women', 'fgm', 'early marriage', \
                            'forced marriage', 'sexual mutilation', 'genital mutilation', 'sexual exploitation']

# Civil and Political Rights

keywords['speech'] = ['speech','expression','journalis','opinion',' press ',' media','insult law',\
                      'defamation','political dissent','newspaper','censor', 'legislation communications']

keywords['religion'] = ['religio', 'church', 'muslim', 'blasphemy', 'apostasy', 'freedom belief', \
                        'conscientious objection', 'freedom conscience']

keywords['domest-movement'] =['resettlement', 'checkpoints', 'relocat' , 'internal exile', 'travel permit', \
                       'identification document', 'freedom movement', 'internal displacement', 'forced displacement'] 

keywords['foreign-movement'] = ['passport', 'emigrat', 'exit visa', 'repatriation', 'travel ban', \
                                'foreign travel', 'travel document', 'travel freely', 'unauthorized travel', \
                                'travel abroad']

keywords['civil-so'] = ['civil society','rights defenders','charities','protest','activist','demonstrator', \
                       'freedom assembly','freedom association','free association','free assembly','demonstrations' \
                       'political party', 'union member','governmental organisations', 'governmental organization' \
                       ]


keywords['self-determ'] =['democ', 'electi', 'referendum', 'self-determination', 'self determination', \
                          'opposition parti','political parti']

keywords['judiciary'] = ['judiciary', 'judicial', 'fair trial', 'judge', 'defendants', 'prosecutor', \
                         'lawyer', 'legal representation', 'legal aid', 'due process']

keywords['privacy'] = ['privacy']

keywords['property'] = ['property','properties','inheritance']

keywords['discrimination'] = ['discriminat', 'stereotyp', 'attitude', 'role responsibilit', \
                              'equal', 'segregation']

keywords['reproductive'] = ['reproductive', 'abortion', 'birth control', 'family planning', 'pregnancy']

keywords['governance'] = ['good governance', 'rule law', 'public governance']

keywords['corruption'] = ['corruption','bribary','bribe','bribing','extortion']

# Social, Cultural and Economic Rights

keywords['labor'] = [' labor', 'labour', 'strike', 'unions', 'worker', 'employment', \
                     'collective bargain', 'occupational', 'wage']


keywords['poverty'] = ['poverty', 'social security', 'pension', 'social safety net', 'living standard', \
                       'unemployment', 'social welfare', 'public service', 'job creation', 'right development', \
                      'economic development', 'economic exploitation', 'human development', 'pro-poor growth', \
                      'social programmes', 'global financial crisis', 'development effort', \
                      'mgd', 'development goal', 'economic crisis', 'sustainable economic', 'industr']
                     
keywords['health'] = ['health', 'hospital', 'doctor', 'medical', 'disease', 'vaccination', 'maternal mortality']
                     
keywords['hiv'] = ['hiv']
                     
keywords['food'] = ['food', 'hungry', 'hunger']

keywords['education'] = ['education','illiter','school','academic']
        
keywords['infrastructure'] = ['water', 'sanitation', 'housing', 'roads', 'homeless', 'squatting', 'lodging']


keywords['land'] = ['land rights']

keywords['culture'] = ['cultural rights', 'cultural values', 'traditions', 'cultural social rights']

keywords['environment'] = ['environment', 'pollution', 'climate change', 'climate challenge', 'disaster', \
                          'natural catastrophe']

keywords['socialism'] = ['socialism', 'communism']

# Conflict

keywords['post-conflict'] = ['truth commission', 'reconstruction', 'post-conflict', 'reconciliation', 'amnesty',\
                        'post conflict', 'international criminal court']

keywords['civil-conflict'] = ['civil war', 'civil conflict', 'armed conflict', 'war crime', 'crime humanity', \
                              'genocide', 'massacre', 'secessionists']

keywords['terrorism'] = ['terroris']
                              

# Victims
                                                       
keywords['migrants'] = ['migration', 'migrant', 'refugee', 'displaced', 'asylum', 'displacements', 'idps', \
                         'eviction', 'statelessness', 'refoulement']                    

keywords['children'] = ['child', 'juvenile', 'minors', 'girl', 'minimum age', 'age criminal responsibility', \
                       'corporal punishment', 'adolescent']

keywords['lgbt'] = ['lgbt', 'lesbian', 'gay', 'homosexual', 'transsexual', 'sexual orientation', 'gender identity',\
                    'consensual sex', 'same-sex', 'same sex',' homophobia', 'between consenting adults']

keywords['ethnic'] = ['roma','ethnic','minorit', 'xenophob', 'anti-semit', 'islamophob', 'pygmy']

keywords['indigenous'] = ['indigenous', 'aboriginal', 'tribal']

keywords['race'] = ['race', 'racial', 'racis', 'racist'] 

keywords['disabilities'] = ['disabilit','disabled']

keywords['women'] = ['women', 'gender', 'misogyn', 'widows', 'girl', 'maternal','FGM', 'genital mutilation']

keywords['elderly'] = ['elderly', 'pensioner', 'older person']

keywords['poor'] = ['poorest', 'poor people', 'poor households', 'poor marginalized groups', \
                    'poor backgrounds', 'poor families','poorer segments', 'poor population', 'rural poor', \
                   'low-income families', 'who are poor', 'poor sections population', 'poor socioeconomic background' \
                   'poor rural', 'poor person', 'poor population']

# Agents of the State

keywords['police'] = ['police', 'law-enforcement', 'law enforcement']

keywords['military'] = ['military', 'security force', 'army', 'navy', 'special forces', 'republican guard', \
                       'security operations','security agent', 'security sector']

keywords['paramilitary'] = ['paramil']

keywords['intelligence'] = ['intelligence']

# human rights violations
keywords['impunity'] = ['impunity', 'immunity from prosecution', 'brought book', 'bring justice', 'accountability', \
                       'brought justice']

In [32]:
print (keywords.keys())

['death-pen', 'intelligence', 'prisoners', 'elderly', 'culture', 'race', 'education', 'children', 'civil-so', 'disappearances', 'police', 'hiv', 'privacy', 'discrimination', 'impunity', 'governance', 'labor', 'environment', 'religion', 'foreign-movement', 'health', 'speech', 'domest-movement', 'terrorism', 'paramilitary', 'corruption', 'sex-violence', 'poor', 'civil-conflict', 'infrastructure', 'torture', 'food', 'extra-judicial', 'judiciary', 'indigenous', 'lgbt', 'self-determ', 'poverty', 'ethnic', 'women', 'land', 'reproductive', 'post-conflict', 'disabilities', 'migrants', 'pol-imprisonment', 'socialism', 'trafficking', 'military', 'property']


In [33]:
#apply
for i in recs:
    i['Theme'] = []
    text = noStop(i['Text'])
    # get rid of country, so that 'democrat' doesn't catch "democratic republic of congo" etc.
    if "(" in text and ")" in text:
        country = text.split('(')[-1].split(')')[0]
        text = text.replace(country,'')
    for key in keywords.keys():
            for word in keywords[key]:
                if key in i['Theme']: # skip if it's already in there
                    continue
                if word in text:
                    i['Theme'].append(key)

In [34]:
# Special Rules

#"NGO" (capitalization required)
for i in recs:
    if 'civil-so' not in i['Theme']:
        if "NGO" in i['Text']:
            i['Theme'].append('civil-so')
            
# HIV (capitalization required)
for i in recs:
    if 'hiv' not in i['Theme']:
        if "AIDS" in i['Text']:
            i['Theme'].append('hiv')

#"THE poor" (stopword required)
for i in recs:
    if 'poor' not in i['Theme']:
        if "the poor" in i['Text']:
            i['Theme'].append('poor')

# "courts" and "tribunal" but not ICC, ICT, European court etc (lowercase requried)
for i in recs:
    if 'judiciary' not in i['Theme']:
        if 'court' in i['Text'] or 'tribunal' in i['Text']:
            i['Theme'].append('judiciary')
    
# in custody (stop words required)
for i in recs:
    if 'prisoners' not in i['Theme']:
        if 'in custody' in i['Text']:
            i['Theme'].append('prisoners')
            
# assembly = civil society (lowercase requried)
for i in recs:
    if 'civil-so' not in i['Theme']:
        if 'assembly' in i['Text']:
            i['Theme'].append('civil-so')
        
# if sex-violence, then women
for i in recs:
    if 'sex-violence' in i['Theme']:
        if 'women' not in i['Theme']:
            i['Theme'].append('women')

In [35]:
# differentiating women's rights
keywords = {}
keywords['women-social'] = ['child custody', 'divorce', 'personal legal status', 'personal status', 'inheritance']
keywords['women-political'] = ['political', 'decision-making', 'decision making', 'public sphere']
keywords['women-economic'] = [' labor', 'labour', 'worker', 'employment', 'occupational', 'wage', 'property',\
                              'business', 'financial', 'economic']
for i in recs:
    if 'women' in i['Theme']:
        text = noStop(i['Text'])
        for key in keywords.keys():
            for word in keywords[key]:
                if key in i['Theme']: # skip if it's already in there
                    continue
                if word in text:
                    i['Theme'].append(key)


In [36]:
## linking institutions to themes
keywords['women'] = ['CEDAW', 'eu-vaw', 'pol-women']
keywords['sex-violence'] = ['eu-vaw', 'eu-child']
keywords['children'] = ['CRC', 'eu-child']
keywords['torture'] = ['CAT']
keywords['disappearances'] = ['CPED']
keywords['disabilities'] = ['CRPD']
keywords['migrants'] = ['ICMW','eu-migrant','UNHCR']
keywords['race'] = ['ICERD']
keywords['labor'] = ['ILO', 'ICMW']
keywords['trafficking'] = ['traffick', 'eu-traff']
keywords['indigenous'] = ['ind169','un-indig']
keywords['domest-movement'] = ['kampala']
keywords['ethnic'] = ['frame-minorities']
keywords['education'] = ['unesco-ed']
keywords['culture'] = ['unesco-expr']
keywords['post-conflict'] = ['ICC']

for i in recs:
    for key in keywords.keys():
        if key not in i['Theme']:
            for institution in keywords[key]:
                if institution in i['Institution']:
                    i['Theme'].append(key)

In [37]:
# count the themes
themes = []
for rec in recs:
    for i in rec['Theme']:
        themes.append(i)
Counter(themes)

Counter({'women': 7840, 'children': 6030, 'discrimination': 4926, 'education': 3157, 'sex-violence': 2914, 'labor': 2474, 'migrants': 2350, 'torture': 2258, 'ethnic': 1938, 'judiciary': 1807, 'prisoners': 1806, 'speech': 1756, 'civil-so': 1604, 'health': 1574, 'trafficking': 1533, 'death-pen': 1517, 'disabilities': 1440, 'poverty': 1388, 'police': 1123, 'race': 1111, 'religion': 937, 'disappearances': 877, 'post-conflict': 849, 'lgbt': 827, 'impunity': 715, 'culture': 708, 'women-economic': 690, 'indigenous': 652, 'self-determ': 578, 'military': 525, 'infrastructure': 485, 'women-political': 406, 'environment': 318, 'civil-conflict': 269, 'food': 257, 'hiv': 244, 'corruption': 216, 'reproductive': 214, 'governance': 203, 'terrorism': 191, 'property': 166, 'extra-judicial': 158, 'elderly': 113, 'domest-movement': 111, 'women-social': 104, 'poor': 86, 'pol-imprisonment': 59, 'foreign-movement': 33, 'privacy': 31, 'land': 18, 'intelligence': 17, 'paramilitary': 17, 'socialism': 5})

## Explore

In [38]:
# exploring

def topThemes(country):
    allThemes = []
    for i in recs:
        if i['To'] == country:
            if i['Theme'] != []:
                for theme in i['Theme']:
                    allThemes.append(theme)
    return(Counter(allThemes))

In [39]:
topThemes('haiti')

Counter({'children': 35, 'women': 23, 'trafficking': 14, 'prisoners': 12, 'education': 11, 'judiciary': 11, 'post-conflict': 10, 'police': 9, 'discrimination': 9, 'labor': 9, 'sex-violence': 9, 'torture': 8, 'migrants': 8, 'disabilities': 7, 'culture': 6, 'poverty': 6, 'impunity': 5, 'health': 5, 'food': 5, 'women-economic': 4, 'infrastructure': 4, 'disappearances': 3, 'women-political': 2, 'governance': 2, 'domest-movement': 2, 'civil-so': 1, 'corruption': 1, 'self-determ': 1})

In [40]:
def getTheme(country,theme):
    allRecs = []
    for i in recs:
        if i['To'] == country:
            if theme in i['theme']:
                allRecs.append(i['Text'])
    return(allRecs)

In [41]:
getTheme('unitedstatesofamerica','women')

[]

## Write CSV

In [42]:
#writing column headings
keys = recs[0].keys()
keys

['From_COW',
 'Action_Verb',
 'Session_Year',
 'To_COW',
 'Text',
 'Decision',
 'Report_Year',
 'To',
 'Theme',
 'Session',
 'From',
 'Institution',
 'UID']

In [43]:
#writing the rest
with open('../Data/upr-orig.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(recs)